In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2
from collections import deque

In [2]:
train = pd.read_csv('train.csv')
train_copy = train.copy()

In [3]:
def Correct_Square(train_copy):
    
    
    null_list = train_copy.loc[train_copy.LifeSquare.isnull()].index   
   
    for i in null_list:
        
        train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square'] - (train_copy.loc[i]['Square'] / 100 * 15) - \
                                        train_copy.loc[i]['KitchenSquare']
    
    
    ID_List = train_copy.loc[train_copy['LifeSquare'] > train_copy['Square']].index
    for i in ID_List:
        temp = train_copy.loc[i]['LifeSquare']
        train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square']
        train_copy.loc[i, 'Square'] = temp
    
    
    indexoid = train_copy.loc[train_copy.Square - train_copy.LifeSquare > train_copy.LifeSquare * 2].index
    for i in indexoid:
        train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square'] - (train_copy.loc[i]['Square'] / 100 * 20) 

In [4]:
def Correct_floors(train_copy):
    ID_List = []
    ID_List.extend(train_copy.loc[train_copy['Floor'] > train_copy['HouseFloor']].index)
    for i in ID_List:
        if train_copy.loc[i]['Floor'] > -1:
            temp = train_copy.loc[i]['Floor']
            train_copy.loc[i, 'Floor'] = train_copy.loc[i]['HouseFloor']
            train_copy.loc[i, 'HouseFloor'] = temp

In [5]:
def Correct_other(train_copy):
    train_copy.loc[train_copy['HouseYear'] > 2019, 'HouseYear'] = train_copy['HouseYear'].median()
    train_copy.loc[train_copy.KitchenSquare > train_copy.Square, 'KitchenSquare'] = train_copy.loc[train_copy.KitchenSquare > train_copy.Square]['Square'] - train_copy.loc[train_copy.KitchenSquare > train_copy.Square]['LifeSquare']
    train_copy.loc[train_copy.HouseFloor > 50, 'HouseFloor'] = train_copy.loc[train_copy.HouseFloor > 50]['Floor'].median() + 10
    train_copy.loc[(train_copy.Rooms > 5) & (train_copy.loc[train_copy.Rooms > 5]['Square'] < train_copy.loc[train_copy.Rooms == 5]['Square'].mean()), 'Rooms'] = train_copy.loc[train_copy.Rooms]['Rooms'].median() 
    train_copy.loc[train_copy.Square > (train_copy.loc[train_copy.Rooms, 'Square'].mean() + 100), 'Square'] = train_copy.loc[train_copy.Rooms, 'Square'].mean()
    train_copy.drop('Healthcare_1', axis=1, inplace=True)
    
    train_copy.loc[train_copy.Square - train_copy.LifeSquare < train_copy.KitchenSquare, 'LifeSquare'] = \
    train_copy.loc[train_copy.Square - train_copy.LifeSquare < train_copy.KitchenSquare]['Square'] - \
    train_copy.loc[train_copy.Square - train_copy.LifeSquare < train_copy.KitchenSquare]['KitchenSquare']
    
    

In [6]:
features = ['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2',
       'Shops_1']
target = ['Price']

In [7]:
 def prepare_features(train_copy):
    
    Correct_floors(train_copy)
    Correct_other(train_copy)
    Correct_Square(train_copy)


In [8]:
prepare_features(train_copy)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(train_copy[features], train_copy[target], \
                                                      test_size=0.3, random_state=42)

In [10]:
model = RFR(n_estimators=5, max_depth=12, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
r2(y_valid, y_pred)

0.6581623386589152

In [11]:
def Testing_models():
    
    r2_value = 0
    estimator = 0
    depth = 0
    
    for n_estimators in range(1, 16):
        for max_depth in range(1, 25, 2):
            model = RFR(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            if r2_value < r2(y_valid, y_pred):
                r2_value = r2(y_valid, y_pred)
                estimator = n_estimators
                depth = max_depth
           
    
    return estimator, depth, r2_value

In [12]:
def Testing_models_extended(X_train, X_valid, y_train, y_valid):
    
    r2_value = 0
    estimator = 0
    depth = 0
    
    for n_estimators in range(1, 16):
        for max_depth in range(1, 25, 2):
            model = RFR(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            if r2_value < r2(y_valid, y_pred):
                r2_value = r2(y_valid, y_pred)
                estimator = n_estimators
                depth = max_depth
    
    
    
    return estimator, depth, r2_value

In [13]:
n_estimator, max_depth, r2_value = Testing_models()
n_estimator, max_depth, r2_value

(15, 13, 0.6967634183834759)

In [14]:
best_r2_yet = r2_value

In [15]:
final_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [16]:

def Testing_features(features, best_r2_yet):
    
    test_features = deque()
    
# отбор по возрастанию, но менее эффективный    
    
#     r2_best = -100
#     best_feat_list = [None for i in range(len(features))]
#     test_features = features.copy()
#     final_list = []
    
    
#     for i in range(len(test_features)):
        
#         best_feat = None
        
#         for j in range(len(test_features)):
            
#             if not(test_features[j] in best_feat_list):
                
#                 best_feat_list[i] = test_features[j]

#             else:
#                 continue
            
#             final_model.fit(X_train[best_feat_list[:i+1]], y_train)
#             y_pred = final_model.predict(X_valid[best_feat_list[:i+1]])
            
#             if r2_best < r2(y_valid, y_pred):
#                 r2_best = r2(y_valid, y_pred)
#                 best_feat = test_features[j]

        
#         if best_feat != None:
# #             print(best_feat, i)
#             print(r2_best)
#             best_feat_list[i] = best_feat
#             final_list.append(best_feat)
       
        
#     return final_list, r2_best


# отбор по убыванию

    
    flag = True
    r2_best = 0
    test_features = deque(features.copy())
    del_list = []
    must_del_feat = None
    best_r2_yet = best_r2_yet
    
    while flag:
        
        for i in range(len(test_features)):
            
            del_list.append(test_features.pop())
            
            final_model.fit(X_train[test_features], y_train)
            y_pred = final_model.predict(X_valid[test_features])
            
            if r2_best < r2(y_valid, y_pred):
                
                r2_best = r2(y_valid, y_pred)
                
                must_del_feat = del_list[0]
                
            test_features.appendleft(del_list.pop())
        
        if best_r2_yet < r2_best:
            
            best_r2_yet = r2_best
            test_features.remove(must_del_feat)
            
        else:
            flag = False
                
    
    
    return test_features, best_r2_yet

In [17]:

Testing_features(features, best_r2_yet)


(deque(['Square',
        'LifeSquare',
        'KitchenSquare',
        'HouseFloor',
        'HouseYear',
        'Ecology_1',
        'Social_1',
        'Social_2',
        'Social_3',
        'Helthcare_2']),
 0.7183043675097857)

In [18]:
all_feattures = train_copy[train_copy.columns[:-1]].copy()

In [19]:
all_feattures = pd.get_dummies(all_feattures)

In [20]:
all_X_train, all_X_valid, all_y_train, all_y_valid = train_test_split(all_feattures, train_copy['Price'], \
                                                      test_size=0.3, random_state=42)

all_feat_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [21]:
all_feat_model.fit(all_X_train, all_y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=15,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [22]:

all_y_predict = all_feat_model.predict(all_X_valid)
all_r2_best_yet = r2(all_y_valid, all_y_predict)
all_r2_best_yet

0.6899919085049295

In [23]:
def Test_all_feats(features, best_r2_yet):
    flag = True
    r2_best = best_r2_yet
    test_features = deque(features.copy())
    del_list = []
    must_del_feat = None
    final_list = []
    
    
    while flag:
        
        for i in range(len(test_features)):
            
            
            del_list.append(test_features.pop())
            
            all_X_train, all_X_valid, all_y_train, all_y_valid = train_test_split(all_feattures[test_features], train_copy[target], \
                                                      test_size=0.3, random_state=42)

            all_feat_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)
    
            
            
            all_feat_model.fit(all_X_train, all_y_train)
            
            y_pred = all_feat_model.predict(all_X_valid)
    
    
    
            
            if r2_best < r2(all_y_valid, y_pred):
                
                r2_best = r2(all_y_valid, y_pred)
                
                must_del_feat = del_list[0]
                
                print(r2(all_y_valid, y_pred))
                print(len(test_features))
                print(test_features)
                final_list = test_features.copy()
                print()
                
            test_features.appendleft(del_list.pop())
        
        if best_r2_yet < r2_best:
            
            best_r2_yet = r2_best
            test_features.remove(must_del_feat)
                
            
        else:
            flag = False
                

    
    return final_list, best_r2_yet, len(test_features), 

In [24]:
optimal_features, R2_score_, feat_list_len = Test_all_feats(all_feattures, all_r2_best_yet)
optimal_features

0.6929937952128231
20
deque(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A'])

0.6942837588886224
20
deque(['Shops_2_B', 'Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B'])

0.6971043667075933
20
deque(['Ecology_3_B', 'Shops_2_A', 'Shops_2_B', 'Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B'])

0.6974855588797736
20
deque(['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Soci

deque(['Ecology_3_B',
       'Shops_2_B',
       'DistrictId',
       'Square',
       'LifeSquare',
       'KitchenSquare',
       'HouseFloor',
       'HouseYear',
       'Ecology_1',
       'Social_1',
       'Social_2',
       'Social_3',
       'Helthcare_2',
       'Ecology_2_A',
       'Ecology_2_B'])

In [25]:
R2_score_, feat_list_len

(0.716613047418647, 15)

In [26]:
train_last_copy = train.copy()
train_last_copy = pd.get_dummies(train_last_copy)
optimal_features = list(optimal_features)
prepare_features(train_last_copy)
X_train_last, X_valid_last, y_train_last, y_valid_last = train_test_split(train_last_copy[optimal_features], train_last_copy[target], \
                                                      test_size=0.3, random_state=42)

In [27]:
n_estimator, max_depth, r2_value = Testing_models_extended(X_train_last, X_valid_last, y_train_last, y_valid_last)

In [28]:
n_estimator, max_depth, r2_value

(14, 13, 0.7169341061704311)

### финализируем

In [29]:
test = pd.read_csv('test.csv')
test_copy = test.copy()

In [30]:
test_copy = pd.get_dummies(test_copy)

In [31]:
prepare_features(test_copy)

In [32]:
test_copy.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,8412.595400,51.279200,1.906200,56.210653,39.483573,5.830664,7.838000,13.371600,1984.315200,0.119874,...,5406.900000,8.262600,1.31940,4.242800,0.009600,0.990400,0.029800,0.970200,0.082400,0.917600
std,4832.674037,44.179466,0.808908,18.334214,16.763178,4.644416,5.454989,6.053597,18.514092,0.120070,...,4026.614773,23.863762,1.47994,4.777365,0.097518,0.097518,0.170052,0.170052,0.275001,0.275001
min,1.000000,0.000000,0.000000,1.378543,0.975769,0.000000,0.000000,1.000000,1908.000000,0.000000,...,168.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,27.232598,1.000000,4.000000,9.000000,1973.000000,0.019509,...,1564.000000,0.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
50%,8320.500000,37.000000,2.000000,52.811661,35.667465,6.000000,7.000000,14.000000,1977.000000,0.072158,...,5285.000000,2.000000,1.00000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
75%,12598.250000,77.000000,2.000000,66.225831,48.377008,9.000000,11.000000,17.000000,2000.000000,0.195781,...,7287.000000,5.000000,2.00000,6.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,16795.000000,212.000000,6.000000,141.540215,136.921090,112.000000,46.000000,48.000000,2019.000000,0.521867,...,19083.000000,141.000000,6.00000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:

test_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [34]:
test_model.fit(X_train_last, y_train_last)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=14,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [35]:
test_model_pred = test_model.predict(X_valid_last)

In [36]:
r2(y_valid_last, test_model_pred)

0.7169341061704311

In [37]:
test_final = test_model.predict(test_copy[optimal_features])

In [38]:
document = pd.DataFrame({'Id': test_copy['Id'], 'Price': test_final})
document.to_csv('Artem_Minasyan_predictions.csv', index=None)

In [39]:
document.head()

,Id,Price
0,725,154686.966833
1,15856,232548.135066
2,5480,172401.038017
3,15664,378635.066023
4,14275,144656.205283
